In [7]:
import pandas as pd

In [11]:
def load_berrar_ratings():
    file_pattern = 'berrar_ratings/data_recent_and_val_*.csv'
    files = glob.glob(file_pattern)

    full_df = pd.DataFrame()

    for file in files:
        df = pd.read_csv(file)
        # change the format of the Date column
        df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
        full_df = pd.concat([full_df, df])

    # drop the Sea, Lge, GD, WDL columns
    full_df = full_df.drop(columns=['Sea', 'Lge', 'GD', 'WDL'])

    return full_df

In [13]:
df = load_berrar_ratings()

In [14]:
df["Date"] = pd.to_datetime(df["Date"])

In [15]:
df= df.reset_index().set_index(["HT","Date","AT"]).sort_index()

In [16]:
feature_list = [
                "HT_H_Off_Rating",
                "HT_H_Def_Rating",
                "HT_A_Off_Rating",	
                "HT_A_Def_Rating",	
                "AT_H_Off_Rating",	
                "AT_H_Def_Rating",
                "AT_A_Off_Rating",	
                "AT_A_Def_Rating",
                "HS",
                "AS"]

In [17]:
full_data_df_X = df[feature_list]
full_data_df_X = full_data_df_X.reset_index().set_index(["Date", "AT"])
# full_data_df_X.shift(3)

In [18]:

def transform_dataframe(df, n_stagger):
    # Initialize the output DataFrame
    # Shift features 1-7 by 5 time steps (T-5)
    combined = df.copy()
    col_ordered = list(df.columns)
    for i in range(n_stagger):
        T_shift = i+1
        shifted = df.shift(T_shift)
        column_names= [f"{col} T-{T_shift}" for col in df]
        shifted.columns = column_names
        col_ordered = column_names + col_ordered 
        if T_shift == 1: 
            combined = pd.merge(df, shifted, left_index=True, right_index=True)
        else:
            combined = pd.merge(combined, shifted, left_index=True, right_index=True)
    return combined[col_ordered]
def create_seq_group_by(df, features_list,n_stagger):
    list_of_df = [] 
    for i in df.HT.unique(): 
        temp = df[df["HT"] ==i]
        features_only = temp[features_list]
        seq_df = transform_dataframe(features_only, n_stagger)
        seq_df["HT"] = i 
        seq_df = seq_df.reset_index().set_index(["Date","HT", "AT"])
        list_of_df.append(seq_df)
        # break
    return list_of_df

In [19]:
seq_full_data_list_x = create_seq_group_by(full_data_df_X,feature_list,5)


In [20]:
seq_full_data_list_x_cat  = pd.concat(seq_full_data_list_x, axis=0)
seq_full_data_list_x_cat


,,,HT_H_Off_Rating T-5,HT_H_Def_Rating T-5,HT_A_Off_Rating T-5,HT_A_Def_Rating T-5,AT_H_Off_Rating T-5,AT_H_Def_Rating T-5,AT_A_Off_Rating T-5,AT_A_Def_Rating T-5,HS T-5,AS T-5,...,HT_H_Off_Rating,HT_H_Def_Rating,HT_A_Off_Rating,HT_A_Def_Rating,AT_H_Off_Rating,AT_H_Def_Rating,AT_A_Off_Rating,AT_A_Def_Rating,HS,AS
Date,HT,AT,,,,,,,,,,,,,,,,,,,,,
2019-07-26,AC Ajaccio,Havre AC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2,2
2019-08-09,AC Ajaccio,Caen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.663051,0.179601,-0.077462,-1.278073,-0.386057,0.179601,-0.355141,-1.278073,1,2
2019-08-23,AC Ajaccio,Paris FC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.297641,0.362965,-0.145507,-2.476301,-1.796741,-0.135820,-0.416684,2.048477,1,0
2019-09-13,AC Ajaccio,Orleans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.168878,0.045547,-0.216526,-0.916044,-4.102422,-0.205011,-0.208737,-0.098709,0,0
2019-09-27,AC Ajaccio,Valenciennes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.595037,-0.269528,0.539181,-0.287489,-1.289221,-0.742252,-0.555357,-3.183030,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-13,Zwolle,Feyenoord Rotterdam,-12.917282,-2.692460,-6.999407,7.677103,-4.234207,5.623141,-6.322088,4.463554,0.0,1.0,...,-13.477910,-3.080512,-6.918382,4.184365,10.823824,-8.333529,4.706650,-8.888135,1,2
2022-04-03,Zwolle,Go Ahead Eagles,-14.527840,-2.760152,-6.999407,7.677103,-5.905234,-3.696835,-1.815066,0.204811,1.0,3.0,...,-13.126918,-2.986139,-8.040868,3.387887,-1.012439,-0.128373,-6.449985,-2.320938,0,1
2022-04-10,Zwolle,AZ Alkmaar,-14.534697,-1.784190,-8.164636,6.881131,-11.973694,0.768943,-4.465850,7.623807,2.0,0.0,...,-14.361516,-3.039095,-8.040868,3.387887,9.449039,-7.832600,4.402911,-8.793768,2,1


In [21]:
# zero the nan values
seq_full_data_list_x_cat = seq_full_data_list_x_cat.fillna(0)
seq_full_data_list_x_cat

,,,HT_H_Off_Rating T-5,HT_H_Def_Rating T-5,HT_A_Off_Rating T-5,HT_A_Def_Rating T-5,AT_H_Off_Rating T-5,AT_H_Def_Rating T-5,AT_A_Off_Rating T-5,AT_A_Def_Rating T-5,HS T-5,AS T-5,...,HT_H_Off_Rating,HT_H_Def_Rating,HT_A_Off_Rating,HT_A_Def_Rating,AT_H_Off_Rating,AT_H_Def_Rating,AT_A_Off_Rating,AT_A_Def_Rating,HS,AS
Date,HT,AT,,,,,,,,,,,,,,,,,,,,,
2019-07-26,AC Ajaccio,Havre AC,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2,2
2019-08-09,AC Ajaccio,Caen,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.663051,0.179601,-0.077462,-1.278073,-0.386057,0.179601,-0.355141,-1.278073,1,2
2019-08-23,AC Ajaccio,Paris FC,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.297641,0.362965,-0.145507,-2.476301,-1.796741,-0.135820,-0.416684,2.048477,1,0
2019-09-13,AC Ajaccio,Orleans,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,-0.168878,0.045547,-0.216526,-0.916044,-4.102422,-0.205011,-0.208737,-0.098709,0,0
2019-09-27,AC Ajaccio,Valenciennes,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,-1.595037,-0.269528,0.539181,-0.287489,-1.289221,-0.742252,-0.555357,-3.183030,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-13,Zwolle,Feyenoord Rotterdam,-12.917282,-2.692460,-6.999407,7.677103,-4.234207,5.623141,-6.322088,4.463554,0.0,1.0,...,-13.477910,-3.080512,-6.918382,4.184365,10.823824,-8.333529,4.706650,-8.888135,1,2
2022-04-03,Zwolle,Go Ahead Eagles,-14.527840,-2.760152,-6.999407,7.677103,-5.905234,-3.696835,-1.815066,0.204811,1.0,3.0,...,-13.126918,-2.986139,-8.040868,3.387887,-1.012439,-0.128373,-6.449985,-2.320938,0,1
2022-04-10,Zwolle,AZ Alkmaar,-14.534697,-1.784190,-8.164636,6.881131,-11.973694,0.768943,-4.465850,7.623807,2.0,0.0,...,-14.361516,-3.039095,-8.040868,3.387887,9.449039,-7.832600,4.402911,-8.793768,2,1


In [22]:
def save_to_csv(df, filename):
    df.to_csv(filename, index=True)

In [23]:
filename = "dataset/seq_target_data.csv"
save_to_csv(seq_full_data_list_x_cat, filename)